In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, isnan, when, percentile_approx

In [2]:
spark = SparkSession.builder.appName("AirQualityAnalysisIndia").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/19 15:30:49 WARN Utils: Your hostname, Karthikeya, resolves to a loopback address: 127.0.1.1; using 10.146.6.37 instead (on interface wlp1s0)
25/10/19 15:30:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/19 15:30:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.csv('file:///home/karthikeya/Desktop/sem5/MIT_SEM5_BDA/MiniProject/datasets/air_quality_data/city_day.csv', header=True, inferSchema=True)

In [4]:
df.show(5)
print("Rows:", df.count())
print("Columns:", len(df.columns))

+---------+----------+----+----+----+-----+-----+----+----+-----+------+-------+-------+------+----+----------+
|     City|      Date|PM25|PM10|  NO|  NO2|  NOx| NH3|  CO|  SO2|    O3|Benzene|Toluene|Xylene| AQI|AQI_Bucket|
+---------+----------+----+----+----+-----+-----+----+----+-----+------+-------+-------+------+----+----------+
|Ahmedabad|2015-01-01|NULL|NULL|0.92|18.22|17.15|NULL|0.92|27.64|133.36|    0.0|   0.02|   0.0|NULL|      NULL|
|Ahmedabad|2015-01-02|NULL|NULL|0.97|15.69|16.46|NULL|0.97|24.55| 34.06|   3.68|    5.5|  3.77|NULL|      NULL|
|Ahmedabad|2015-01-03|NULL|NULL|17.4| 19.3| 29.7|NULL|17.4|29.07|  30.7|    6.8|   16.4|  2.25|NULL|      NULL|
|Ahmedabad|2015-01-04|NULL|NULL| 1.7|18.48|17.97|NULL| 1.7|18.59| 36.08|   4.43|  10.14|   1.0|NULL|      NULL|
|Ahmedabad|2015-01-05|NULL|NULL|22.1|21.42|37.76|NULL|22.1|39.33| 39.31|   7.01|  18.89|  2.78|NULL|      NULL|
+---------+----------+----+----+----+-----+-----+----+----+-----+------+-------+-------+------+----+----

In [5]:
df.printSchema()

root
 |-- City: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- PM25: double (nullable = true)
 |-- PM10: double (nullable = true)
 |-- NO: double (nullable = true)
 |-- NO2: double (nullable = true)
 |-- NOx: double (nullable = true)
 |-- NH3: double (nullable = true)
 |-- CO: double (nullable = true)
 |-- SO2: double (nullable = true)
 |-- O3: double (nullable = true)
 |-- Benzene: double (nullable = true)
 |-- Toluene: double (nullable = true)
 |-- Xylene: double (nullable = true)
 |-- AQI: double (nullable = true)
 |-- AQI_Bucket: string (nullable = true)



In [6]:
df.describe().show()

25/10/19 15:30:59 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-------------+-----------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+----------+
|summary|         City|             PM25|              PM10|                NO|               NO2|               NOx|               NH3|               CO|               SO2|               O3|           Benzene|           Toluene|            Xylene|               AQI|AQI_Bucket|
+-------+-------------+-----------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+----------+
|  count|        29531|            24933|             18391|             25949|             25946|             25346|             19203|            27472|         

In [7]:
missing_counts = []

for c, dtype in df.dtypes:
    if dtype in ('float', 'double'):
        cond = col(c).isNull() | isnan(col(c))
    else:
        cond = col(c).isNull()
    missing_counts.append(count(when(cond, c)).alias(c))

missing_df = df.select(missing_counts)
missing_df.show()

+----+----+----+-----+----+----+----+-----+----+----+----+-------+-------+------+----+----------+
|City|Date|PM25| PM10|  NO| NO2| NOx|  NH3|  CO| SO2|  O3|Benzene|Toluene|Xylene| AQI|AQI_Bucket|
+----+----+----+-----+----+----+----+-----+----+----+----+-------+-------+------+----+----------+
|   0|   0|4598|11140|3582|3585|4185|10328|2059|3854|4022|   5623|   8041| 18109|4681|      4681|
+----+----+----+-----+----+----+----+-----+----+----+----+-------+-------+------+----+----------+



In [8]:
total_rows = df.count()
threshold = 0.6 * total_rows

missing_counts = missing_df.collect()[0].asDict()

columns_to_drop = [col for col, missing_count in missing_counts.items() if missing_count > threshold]

df_cleaned = df.drop(*columns_to_drop)
print("Dropped columns:", columns_to_drop)

Dropped columns: ['Xylene']


In [9]:
numeric_cols = [field.name for field in df_cleaned.schema.fields if field.dataType.simpleString() in ('int', 'double')]
print(numeric_cols)

median_dict = {}
for col_name in numeric_cols:
    median_val = df_cleaned.select(percentile_approx(col_name, 0.5)).collect()[0][0]
    if median_val is not None:
        median_dict[col_name] = median_val

# Fill missing numeric values
df_filled = df_cleaned.fillna(median_dict)


['PM25', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2', 'O3', 'Benzene', 'Toluene', 'AQI']


In [10]:
df_filled.write.mode("overwrite").parquet("file:///home/karthikeya/Desktop/sem5/MIT_SEM5_BDA/MiniProject/processed/cleaned_air_quality.parquet")

In [11]:
spark.stop()